# Biased coin vs. fair coin

Leonardo Espin, 11/22/2023

I came across the following *conditional probability* problem:

"There are two coins, one is fair and one is biased. The biased coin has a probability of landing on heads equal to 4/5. One of the coins is chosen at random (50-50), and is flipped repeatedly until it lands on tail. If it landed on heads 4 times before landing on tails, what is the posterior probability that coin chosen was biased?"

so, calling the event **HHHHT** alpha, I want to calculate `P(C=B | alpha)` which is  `P(C=B & alpha)/P(alpha)`. now 

`P(alpha) = P(alpha | c=B)*P(c=B) + P(alpha | c=F)*P(c=F)` since 

```
P(alpha) = P(alpha ∩ (c=B ∪ ¬c=B))
         = P((alpha ∩ c=B) ∪ (alpha ∩ ¬c=B))
         = P(alpha ∩ c=B) + P(alpha ∩ ¬c=B)
```


Then, if we replace the values given, we get: 

In [1]:
alpha = 0.5*(((4/5)**4)/5)/(0.5)  +0.5*((1/2)**5)/0.5
alpha

0.11317000000000002

so then  `P(C=B | alpha) =`

In [2]:
(((4/5)**4)/5)/alpha

0.7238667491384643

which seems reasonable, it should be greater than 0.5. but is it correct?

I checked for answers online, and I found diverging answers. For example there is [this one](https://brainly.com/question/13658612), with an explanation that doesn't make sense (130=100!) but is the accepted one.

So, I decided to use a numerical simulation to check my answer:

In [3]:
import random

class Coin():
    '''choose a coin randomly and toss it five times'''
    def __init__(self, ntoss=5):
        self.ntoss = ntoss
        self.coin_choice = ('F', 'B')
        self.fair_toss_dist = ('H', 'T')
        self.fair_len = 2
        self.biased_toss_dist = ('H', 'H', 'H', 'H', 'T')
        self.biased_len = 5
        self.distributions = {'F': self.fair_toss_dist, 'B': self.biased_toss_dist}
        self.lens = {'F': self.fair_len, 'B': self.biased_len}

    
    def choose_coin(self):
        # equal prob. of coin selection
        return self.coin_choice[random.randrange(2)]
    
    def toss_coin(self):
        coin = self.choose_coin()
        out = coin
        for i in range(self.ntoss):
            out += self.distributions[coin][random.randrange(self.lens[coin])]
        return out
    
class GetDist():
    '''run the given number of events and collect statistics'''
    def __init__(self, nevents = 10000):
        self.nevents = nevents
        self.events = []
        self.coin = Coin()
        self.dist = {}
    
    def calculate_dist(self):
        for i in range(self.nevents):
            self.events.append(self.coin.toss_coin())
        
        for x in self.events:
            if x in self.dist:
                self.dist[x] +=1
            else:
                self.dist[x] =1

In [4]:
tosses = GetDist()
tosses.calculate_dist()
tosses.dist

{'FTHHHT': 152,
 'FHHTHH': 154,
 'BHHHTT': 98,
 'BTTHHH': 89,
 'FHTHHT': 156,
 'BHHHHH': 1595,
 'FTHTHT': 139,
 'FTHHTT': 185,
 'BTHHHH': 409,
 'BTTHTH': 26,
 'BHHHHT': 409,
 'BHHHTH': 414,
 'FHTTTH': 160,
 'BTHTHT': 40,
 'BHTHHH': 409,
 'FTTTHT': 154,
 'FTTTTT': 151,
 'FHHTTT': 157,
 'FHHTTH': 141,
 'BHHTHH': 427,
 'FTHTTT': 156,
 'FTHHHH': 177,
 'FHTHTH': 152,
 'BTHHHT': 80,
 'FHTHHH': 188,
 'FTTTHH': 144,
 'FTHTHH': 154,
 'FHTTTT': 165,
 'BTHTHH': 84,
 'FHTTHH': 195,
 'FHHHTT': 164,
 'FTHHTH': 157,
 'BTTHHT': 27,
 'FTTHTH': 178,
 'FTHTTH': 143,
 'BHHTTH': 97,
 'BTHHTH': 94,
 'BTHTTH': 25,
 'FHHTHT': 154,
 'FTTTTH': 161,
 'FHTTHT': 138,
 'FHHHHT': 174,
 'BHTTHH': 110,
 'FTTHTT': 168,
 'BTTTHH': 30,
 'FTTHHT': 143,
 'FHHHHH': 157,
 'BHTTTH': 33,
 'FHHHTH': 168,
 'BHTHTH': 98,
 'BTTTHT': 5,
 'FHTHTT': 145,
 'BHHTTT': 26,
 'FTTHHH': 164,
 'BHTHHT': 91,
 'BTHHTT': 20,
 'BHHTHT': 100,
 'BHTTHT': 25,
 'BHTHTT': 19,
 'BTHTTT': 3,
 'BTTTTH': 8,
 'BTTHTT': 7,
 'BHTTTT': 7,
 'BTTTTT': 1}

so, we can calculate `P(C=B | alpha)` directly:

In [5]:
# p(c=b & alpha)
tosses.dist['BHHHHT']/tosses.nevents

0.0409

In [6]:
# p(c=b | alpha) 
(tosses.dist['BHHHHT']/tosses.nevents)/(tosses.dist['BHHHHT']/tosses.nevents +tosses.dist['FHHHHT']/tosses.nevents)

0.7015437392795884

which is pretty close to the value `0.724` I got above. If I run it again using a larger sample space:

In [7]:
tosses = GetDist(nevents=100000)
tosses.calculate_dist()
# p(c=b | alpha) 
(tosses.dist['BHHHHT']/tosses.nevents)/(tosses.dist['BHHHHT']/tosses.nevents +tosses.dist['FHHHHT']/tosses.nevents)

0.7293865140404221

that's closer!. So I'm pretty confident the answer I calculated above is correct.